### This Notebook shows how a **Backer** can writte KERI Key Events to Cardano blockchain and how a **Watcher** can retrieve the full Key Event Log (KEL) from the blockchain.
This code use Blockfrost API to intercat with Cardano Nodes.

In [1]:
from blockfrost import BlockFrostApi, ApiError, ApiUrls
from pycardano import * 
import os
import json
from cbor2 import dumps, loads

First create a ProjectId in https://blockfrost.io for the network of your choice. In this case we are using the _Preview_ network.
Put your projectId in an evironmental variable or paste it in the code bellow.

In [2]:
blockfrostProjectId = os.getenv('BLOCKFROST_PROJECT_ID')
blockfrostProjectId="previewapifaDDKsMZE7asmrcG8W3zbRE1pojXY"
api = BlockFrostApi(
    project_id=blockfrostProjectId,
    base_url=ApiUrls.preview.value
)
network = Network.TESTNET
context = BlockFrostChainContext(blockfrostProjectId,network, ApiUrls.preview.value)

Generate payment and stake key pairs. You can Save and Load on this disk.

In [3]:
payment_key_pair = PaymentKeyPair.generate()
payment_signing_key = payment_key_pair.signing_key
payment_verification_key = payment_key_pair.verification_key

stake_key_pair = StakeKeyPair.generate()
stake_signing_key = stake_key_pair.signing_key
stake_verification_key = stake_key_pair.verification_key

In [ ]:
# Save keys
payment_signing_key.save("payment.skey")
payment_verification_key.save("payment.vkey")
stake_signing_key.save("stake.skey")
stake_verification_key.save("stake.vkey")

In [3]:
# Load keys
payment_signing_key = PaymentSigningKey.load("payment.skey")
payment_verification_key = PaymentVerificationKey.load("payment.vkey")
stake_signing_key = StakeSigningKey.load("stake.skey")
stake_verification_key = StakeVerificationKey.load("stake.vkey")

Generate the Stake Address and the Spending Address that the Backer will use to submit transactions with metadata to Cardano.
The you can fund the Spending Address with test ADA at https://docs.cardano.org/cardano-testnet/tools/faucet"

In [4]:
spending_addr = Address(payment_verification_key.hash(), stake_verification_key.hash(), network=network)
stake_addr = Address(payment_part=None, staking_part=stake_verification_key.hash(), network=network)
print("Stake address:", stake_addr.encode())
print("Spending Address:", spending_addr.encode())

Stake address: stake_test1ur3vs3x0xl97tt2gau5wknmxve0jlmsqn2l2dqrrt40tgzsmhat72
Spending Address: addr_test1qrtvrpe58csevt7esxqjv4rpvm5q30paghtlse2pj3xn8g0zepzv7d7tukk53megad8kvejl9lhqpx4756qxxh27ks9qqxkex3


Using Blockfrost API you can query ADA balances and UTXOs

In [5]:
# Check address balance
address = api.address(
        address=spending_addr.encode())
for amount in address.amount:
    print(amount.quantity, amount.unit)

99801279 lovelace


In [6]:
# Check wallet balance
addresses = api.account_addresses(stake_addr.encode())
for addr in addresses:
    addrR = api.address(addr.address)
    for amount in addrR.amount:
        print(amount.quantity, amount.unit,":", addrR.address)


99801279 lovelace : addr_test1qrtvrpe58csevt7esxqjv4rpvm5q30paghtlse2pj3xn8g0zepzv7d7tukk53megad8kvejl9lhqpx4756qxxh27ks9qqxkex3


In [8]:
# Get UTXOs
utxos = api.address_utxos(spending_addr.encode())
print(utxos)

[Namespace(tx_hash='313d7e96841d7a7ae29dfb3f55c85847f9b3abc6ac40944c4f2c59dec17ef4b6', tx_index=0, output_index=0, amount=[Namespace(unit='lovelace', quantity='100000000')], block='364045efa5a3360d9126afe551981f97b2445ad7928684c5a3de86cec9c6523e', data_hash=None, inline_datum=None, reference_script_hash=None)]


Build and sign a transaction. The Key Event is added as transaction metadata.

In [26]:
builder = TransactionBuilder(context)
builder.add_input_address(spending_addr)
builder.add_output(TransactionOutput(spending_addr,Value.from_primitive([1000000])))
builder.auxiliary_data = AuxiliaryData(Metadata(
            { 
                1: {
                    "ked": {
                        "v": 0x786c616474676370366e754757863377833676b34326e713861377a3572,
                        "t": "icp",
                        "d": "EJIgGbjnsAnxnfDAPmRXjs2XAZVhBpL5AKxMzTrCtkkY",
                        "i": "EJIgGbjnsAnxnfDAPmRXjs2XAZVhBpL5AKxMzTrCtkkY",
                        "s": "0",
                        "kt": "1",
                        "k": [
                            "DPKv6TIDqfNpeJTKg1QI7Ce_lXickic_fSkunB11JZfs"
                        ],
                        "nt": "1",
                        "n": [
                            "EJYY41UY8bqhi0hjHcmyRndjtOeTETHU29SpX9uRLnbT"
                        ],
                        "bt": "1",
                        "b": [
                            "BDJ_0ChB3hmtzzuuKBowKhSj3HsVDT87dnbaYPMlYgIa"
                        ],
                        "c": [],
                        "a": []
                    },
                    "signatures": [
                        {
                            "index": 0,
                            "signature": ["AAACmhlx_wl2aNyckmgrxfQ-u19PnhTqfmaNm-1w","ETxO75xtV-cxl7g07rwSd8gjcrWAd5S1DJa6vZM6zJt6Oh4P"]
                        }
                    ],
                    "witnesses": [
                        "BDJ_0ChB3hmtzzuuKBowKhSj3HsVDT87dnbaYPMlYgIa"
                    ],
                    "witness_signatures": [
                        {
                            "index": 0,
                            "signature": ["AABkpVx4A6hlpC9LPDBHG9TLXbj86lirhD7xLt6x1","xWR5cPybo4Jo7mFVapWhZMpx1ZxCgyE4A5XlJuLN3EK5TUC"]
                        }
                    ],
                    "receipts": {},
                    "timestamp": "2022-11-09T19:19:45.246677+00:00"
                }
            }
        )
    )
signed_tx = builder.build_and_sign([payment_signing_key], change_address=spending_addr)
print(signed_tx.id)

317b7dc3a13506ed72101f7f9d7d8f385285b51ae8d937eb3a289fd1a0a24eb6


Submit transaction to Cardano

In [27]:
context.submit_tx(signed_tx.to_cbor())

ApiError: {'error': 'Bad Request', 'message': '"transaction read error RawCborDecodeError [DecoderErrorDeserialiseFailure \\"Byron Tx\\" (DeserialiseFailure 1 \\"Size mismatch when decoding TxAux.\\\\nExpected 2, but found 4.\\"),DecoderErrorDeserialiseFailure \\"Shelley Tx\\" (DeserialiseFailure 1 \\"Size mismatch when decoding \\\\nRecord RecD.\\\\nExpected 4, but found 3.\\"),DecoderErrorDeserialiseFailure \\"Shelley Tx\\" (DeserialiseFailure 1 \\"Size mismatch when decoding \\\\nRecord RecD.\\\\nExpected 4, but found 3.\\"),DecoderErrorDeserialiseFailure \\"Shelley Tx\\" (DeserialiseFailure 1 \\"Size mismatch when decoding \\\\nRecord RecD.\\\\nExpected 4, but found 3.\\"),DecoderErrorDeserialiseFailure \\"Shelley Tx\\" (DeserialiseFailure 328 \\"An error occured while decoding metadata.\\\\nError: Unsupported CBOR token type TypeInteger\\"),DecoderErrorDeserialiseFailure \\"S', 'status_code': 400}

Check transaction. It may takes time to get published

In [31]:
tx = api.transaction(signed_tx.id)
print(tx)

Namespace(hash='d3fd1fc97bf963d435186d11d411de70383fdcfed4f24856133bc43746896a9d', block='feca672e624ba0ce7bf18ce8d98b2e009fec436f89b0c34c584104e381b121dd', block_height=63003, block_time=1668028412, slot=1372412, index=0, output_amount=[Namespace(unit='lovelace', quantity='99801279')], fees='198721', deposit='0', size=984, invalid_before=None, invalid_hereafter=None, utxo_count=3, withdrawal_count=0, mir_cert_count=0, delegation_count=0, stake_cert_count=0, pool_update_count=0, pool_retire_count=0, asset_mint_or_burn_count=0, redeemer_count=0, valid_contract=True)


In [32]:
metadata = api.transaction_metadata(signed_tx.id)
print(metadata)

[Namespace(label='1', json_metadata=Namespace(ked=Namespace(a=[], b=['BDJ_0ChB3hmtzzuuKBowKhSj3HsVDT87dnbaYPMlYgIa'], c=[], d='EJIgGbjnsAnxnfDAPmRXjs2XAZVhBpL5AKxMzTrCtkkY', i='EJIgGbjnsAnxnfDAPmRXjs2XAZVhBpL5AKxMzTrCtkkY', k=['DPKv6TIDqfNpeJTKg1QI7Ce_lXickic_fSkunB11JZfs'], n=['EJYY41UY8bqhi0hjHcmyRndjtOeTETHU29SpX9uRLnbT'], s='0', t='icp', v='KERI10JSON000159_', bt='1', kt='1', nt='1'), receipts=Namespace(), timestamp='2022-11-09T19:19:45.246677+00:00', witnesses=['BDJ_0ChB3hmtzzuuKBowKhSj3HsVDT87dnbaYPMlYgIa'], signatures=[Namespace(index=0, signature=['AAACmhlx_wl2aNyckmgrxfQ-u19PnhTqfmaNm-1w', 'ETxO75xtV-cxl7g07rwSd8gjcrWAd5S1DJa6vZM6zJt6Oh4P'])], witness_signatures=[Namespace(index=0, signature=['AABkpVx4A6hlpC9LPDBHG9TLXbj86lirhD7xLt6x1', 'xWR5cPybo4Jo7mFVapWhZMpx1ZxCgyE4A5XlJuLN3EK5TUC'])]))]


The Backer should submit transactions any time it receives a Key Event.
The Watcher can retrieve the full KEL from Cardano blockchain as follows:

In [6]:
txs = api.address_transactions("addr_test1qp6fllu9amjkf835vrva4hg6m6puptgcp6nuvkjqj2s43v4auyjzn8g79sl6pxk0sh9te9nxq4h2tsh4uxc7x3gk42nq8a7z5r")
for tx in txs:
    m = api.transaction_metadata(tx.tx_hash)
    if m: print(m[0].label,":",vars(m[0]))

0 : {'label': '0', 'json_metadata': Namespace(ked=Namespace(a=[], b=['BK9KEdwXuBfnHe2QKJu1hryM7dg9A7EIbDbvwjoN0ChR'], c=[], d='EIfGM24B_KAMcBTeR1vkhTjQtnNWgHoaZY5j3TNyiZ_4', i='EIfGM24B_KAMcBTeR1vkhTjQtnNWgHoaZY5j3TNyiZ_4', k=['DMWFKggdP227rz3SX_p5nWDTw4TmTlvpw-Uv-J207EZ6'], n=['EEBP5Sucqims_QRC3mB6BC0gJdRwPjAeNsma9ltp2yh0'], s='0', t='icp', v='KERI10JSON000159_', bt='1', kt='1', nt='1'), witnesses=['BK9KEdwXuBfnHe2QKJu1hryM7dg9A7EIbDbvwjoN0ChR'], signatures=[['AACPR1PgzGsLL5oQ0idVPjh9aALSw_3nRE591PjuaWEz', '-HY0q0vqAy5ZSgUxZzqoDygK94nwpLs9EDjDphoLpzQM']], witness_signatures=[['AACFmKJKG4WGO8bW_MAX4rJ1BuGNZfGgPx3w-vajRkgC', 'd4Ckfnm7D4uqOn8qZg5OYzZk4oYTyzlK7SHlJtAMFHMJ']])}
1 : {'label': '1', 'json_metadata': Namespace(ked=Namespace(a=[Namespace(blah='blah')], d='EAH27bQ2rqUCGQZv1BqmLknHq3xXrRmO_h_jdAk4bqmj', i='EIfGM24B_KAMcBTeR1vkhTjQtnNWgHoaZY5j3TNyiZ_4', p='EIfGM24B_KAMcBTeR1vkhTjQtnNWgHoaZY5j3TNyiZ_4', s='1', t='ixn', v='KERI10JSON0000da_'), witnesses=[], signatures=[['AABJNkXVpsg

In [5]:
print(vars(m[0]))

{'label': '1', 'json_metadata': Namespace(ked=Namespace(a=[Namespace(blah='blah')], d='EPpChMODktVjz55UEcHBaEhzVzejRrKCMSNX1Gm_1sZl', i='EDL6d6U6a1ZkXZUcjlZpxDzmdAiedHaql9uwjKK3mj1K', p='EDL6d6U6a1ZkXZUcjlZpxDzmdAiedHaql9uwjKK3mj1K', s='1', t='ixn', v='KERI10JSON0000da_'), witnesses=[], signatures=[['AAA-NAV2BOM-GGGxQlqInCcxfRQm35QW4EZJgD7tBwIR', 'bDUETiCTiYdrSBCFBIhMByH2QN75zfIU4VLb21vUCL8F']], witness_signatures=[['AADfA9O_BFAy2PEkkY6Sz-Ko-eJMsmPBT-1wlJCPjqJS', 'yQ7_PIdhQjkQIsLAAhDG1zIDPuldE78cHi_2fOE9CrAN']])}


In [29]:
try:
    payment_signing_key = PaymentSigningKey.load("payment.skey")
except:
    print("coud]ndt load")

In [19]:
ke = {'ked': {'v': 'KERI10JSON000159_', 't': 'icp', 'd': 'EJIgGbjnsAnxnfDAPmRXjs2XAZVhBpL5AKxMzTrCtkkY', 'i': 'EJIgGbjnsAnxnfDAPmRXjs2XAZVhBpL5AKxMzTrCtkkY', 's': '0', 'kt': '1', 'k': ['DPKv6TIDqfNpeJTKg1QI7Ce_lXickic_fSkunB11JZfs'], 'nt': '1', 'n': ['EJYY41UY8bqhi0hjHcmyRndjtOeTETHU29SpX9uRLnbT'], 'bt': '1', 'b': ['BDJ_0ChB3hmtzzuuKBowKhSj3HsVDT87dnbaYPMlYgIa'], 'c': [], 'a': []}, 'stored': True, 'signatures': [{'index': 0, 'signature': 'AAACmhlx_wl2aNyckmgrxfQ-u19PnhTqfmaNm-1wETxO75xtV-cxl7g07rwSd8gjcrWAd5S1DJa6vZM6zJt6Oh4P'}], 'witnesses': ['BDJ_0ChB3hmtzzuuKBowKhSj3HsVDT87dnbaYPMlYgIa'], 'witness_signatures': [{'index': 0, 'signature': 'AABkpVx4A6hlpC9LPDBHG9TLXbj86lirhD7xLt6x1xWR5cPybo4Jo7mFVapWhZMpx1ZxCgyE4A5XlJuLN3EK5TUC'}], 'receipts': {}, 'timestamp': '2022-11-09T19:19:45.246677+00:00'}

In [22]:
print(json.dumps(ke, indent=4))

{
    "ked": {
        "v": "KERI10JSON000159_",
        "t": "icp",
        "d": "EJIgGbjnsAnxnfDAPmRXjs2XAZVhBpL5AKxMzTrCtkkY",
        "i": "EJIgGbjnsAnxnfDAPmRXjs2XAZVhBpL5AKxMzTrCtkkY",
        "s": "0",
        "kt": "1",
        "k": [
            "DPKv6TIDqfNpeJTKg1QI7Ce_lXickic_fSkunB11JZfs"
        ],
        "nt": "1",
        "n": [
            "EJYY41UY8bqhi0hjHcmyRndjtOeTETHU29SpX9uRLnbT"
        ],
        "bt": "1",
        "b": [
            "BDJ_0ChB3hmtzzuuKBowKhSj3HsVDT87dnbaYPMlYgIa"
        ],
        "c": [],
        "a": []
    },
    "stored": true,
    "signatures": [
        {
            "index": 0,
            "signature": "AAACmhlx_wl2aNyckmgrxfQ-u19PnhTqfmaNm-1wETxO75xtV-cxl7g07rwSd8gjcrWAd5S1DJa6vZM6zJt6Oh4P"
        }
    ],
    "witnesses": [
        "BDJ_0ChB3hmtzzuuKBowKhSj3HsVDT87dnbaYPMlYgIa"
    ],
    "witness_signatures": [
        {
            "index": 0,
            "signature": "AABkpVx4A6hlpC9LPDBHG9TLXbj86lirhD7xLt6x1xWR5cPybo4Jo7mFVapWhZM

In [56]:
for wsig in ke['witness_signatures']:
    ke['witness_signatures'][wsig['index']] = [wsig['signature'][:44],wsig['signature'][44:]]

In [ ]:
for sig in ke['signatures']:
    ke['signatures'][sig['index']] = [sig['signature'][:44],sig['signature'][44:]]

In [57]:
print(json.dumps(ke, indent=4))

{
    "ked": {
        "v": "KERI10JSON000159_",
        "t": "icp",
        "d": "EJIgGbjnsAnxnfDAPmRXjs2XAZVhBpL5AKxMzTrCtkkY",
        "i": "EJIgGbjnsAnxnfDAPmRXjs2XAZVhBpL5AKxMzTrCtkkY",
        "s": "0",
        "kt": "1",
        "k": [
            "DPKv6TIDqfNpeJTKg1QI7Ce_lXickic_fSkunB11JZfs"
        ],
        "nt": "1",
        "n": [
            "EJYY41UY8bqhi0hjHcmyRndjtOeTETHU29SpX9uRLnbT"
        ],
        "bt": "1",
        "b": [
            "BDJ_0ChB3hmtzzuuKBowKhSj3HsVDT87dnbaYPMlYgIa"
        ],
        "c": [],
        "a": []
    },
    "stored": true,
    "signatures": [
        [
            "AAACmhlx_wl2aNyckmgrxfQ-u19PnhTqfmaNm-1wETxO",
            "75xtV-cxl7g07rwSd8gjcrWAd5S1DJa6vZM6zJt6Oh4P"
        ]
    ],
    "witnesses": [
        "BDJ_0ChB3hmtzzuuKBowKhSj3HsVDT87dnbaYPMlYgIa"
    ],
    "witness_signatures": [
        [
            "AABkpVx4A6hlpC9LPDBHG9TLXbj86lirhD7xLt6x1xWR",
            "5cPybo4Jo7mFVapWhZMpx1ZxCgyE4A5XlJuLN3EK5TUC"
        ]
    ]

In [21]:
data = dumps("addr_test1qp6fllu9amjkf835vrva4hg6m6puptgcp6nuvkjqj2s43v4auyjzn8g79sl6pxk0sh9te9nxq4h2tsh4uxc7x3gk42nq8a7z5r")
print(data.hex())

786c616464725f7465737431717036666c6c7539616d6a6b6638333576727661346867366d3670757074676370366e75766b6a716a3273343376346175796a7a6e38673739736c3670786b307368397465396e787134683274736834757863377833676b34326e713861377a3572


In [60]:
utxos = api.address_utxos("addr_test1qpyex2dhz597lflafzhusp0t695602flrntxydz3ww5sru4lakj244r7yjsvdwp6pd0hr70vuzkur0a5rtuj6srufa9qz76fy0")

In [61]:
utxo_sum = 0
for u in utxos:
    print(u)
    utxo_sum = utxo_sum + int(u.amount[0].quantity)
    print(u.amount[0].quantity)
    if utxo_sum > 1000000: break

Namespace(tx_hash='fbba949dce28e3a7d68a6ccfe4b09507edf0ce23e5a706ed6750a42feafa1298', tx_index=0, output_index=0, amount=[Namespace(unit='lovelace', quantity='100000000')], block='ba82d6cecdb1ae379ef72c31e12a29c8893cbe98b95895ba94c2c05ec0f05843', data_hash=None, inline_datum=None, reference_script_hash=None)
100000000


In [4]:
payment_key_pair = PaymentKeyPair.generate()
payment_signing_key = payment_key_pair.signing_key
payment_verification_key = payment_key_pair.verification_key

stake_key_pair = StakeKeyPair.generate()
stake_signing_key = stake_key_pair.signing_key
stake_verification_key = stake_key_pair.verification_key

In [8]:
print(payment_key_pair.signing_key)
print(payment_key_pair.verification_key)

{"type": "PaymentSigningKeyShelley_ed25519", "description": "PaymentSigningKeyShelley_ed25519", "cborHex": "58201938167e7c022f8e7e3b96ff88033816e50b03bb0573045f75844ea1b41dfb19"}
{"type": "PaymentVerificationKeyShelley_ed25519", "description": "PaymentVerificationKeyShelley_ed25519", "cborHex": "5820e3bfb4c228ed37822c694da36b53721bb9eb82cb106fe20d4adb593334112a86"}


In [10]:
spending_addr = Address(payment_verification_key.hash(), payment_verification_key.hash(), network=network)
stake_addr = Address(payment_part=None, staking_part=stake_verification_key.hash(), network=network)
print("Stake address:", stake_addr.encode())
print("Spending Address:", spending_addr.encode())

Stake address: stake_test1up4wmlz82984zmpdr52ke3a45fdsp38n97nactlgqpdm60sjhcfmm
Spending Address: addr_test1qp4x53gvwd6d8tw373xapt63ehrdxnp766r90y4s06fmx5m2dfzscum56wkarazd6zh4rnwx6dxra45x27ftql5nkdfszjq5gl


In [11]:
print(payment_verification_key.hash())

6a6a450c7374d3add1f44dd0af51cdc6d34c3ed6865792b07e93b353
